In [1]:
import pandas as pd
import numpy as np
import datetime 
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from scipy import stats

In [2]:
# 데이터 불러오기
weather=pd.read_csv("risk_data/Weather_risk_table_final.csv", encoding='utf-8-sig')
birds=pd.read_csv("risk_data/birds_risk_table.csv", encoding='utf-8-sig')
distance=pd.read_csv("risk_data/distance_risk_final.csv", encoding='utf-8-sig')

In [3]:
weather.head()

,시도,month,day,avg_temp(C),daily_rain(mm),avg_r_humidity(%),temp_index,rain_index,humidity_index,risk
0,강원영동,1,1,1.2,0.0,33.2,0.0,0.0,0.0,0.0
1,강원영동,1,2,3.2,0.0,34.7,0.0,0.0,0.0,0.0
2,강원영동,1,3,3.4,0.0,35.5,0.0,0.0,0.0,0.0
3,강원영동,1,4,4.2,0.0,39.0,0.0,0.0,0.0,0.0
4,강원영동,1,5,3.2,1.8,43.8,0.0,0.0,0.0,0.0


In [4]:
birds.head()

,month,시도,발생건수(타겟),철새수,result,철새수/result,표준 risk,risk
0,1,강원도,2,626130,947733,66%,5,3
1,1,경기도,55,1242626,1072138,116%,5,5
2,1,경상남도,11,347682,968858,36%,5,1
3,1,경상북도,2,554884,947733,59%,5,2
4,1,광주광역시,0,49915,943038,5%,5,0


In [5]:
distance.head()

,month,dist_range,cnt,cumulative,risk
0,1,0,0,0,10
1,1,10,2,2,10
2,1,20,4,6,10
3,1,30,8,14,10
4,1,40,7,21,10


In [6]:
# 전국 시군 데이터 가져오기
all_sigun=pd.read_csv("전국_시군구(완).csv", encoding='utf-8-sig')

In [7]:
all_sigun.head()

,주소,x,y,xy
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813"
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601"
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314"
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337"
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667"


In [8]:
all_sigun['시도']=np.nan
all_sigun['시군구']=np.nan
for i in range(len(all_sigun)):
    temp=all_sigun.loc[i,'주소'].split()
    all_sigun.loc[i,'시도']=temp[0]
    all_sigun.loc[i,'시군구']=temp[1]

In [9]:
all_sigun.head()

,주소,x,y,xy,시도,시군구
0,서울특별시 종로구,126.978990,37.573504,"126.978989954189,37.5735042429813",서울특별시,종로구
1,서울특별시 중구,126.997555,37.563808,"126.997555182293,37.5638077703601",서울특별시,중구
2,서울특별시 용산구,126.990582,37.532431,"126.990582345331,37.5324310391314",서울특별시,용산구
3,서울특별시 성동구,127.036930,37.563427,"127.036930141185,37.563427205337",서울특별시,성동구
4,서울특별시 광진구,127.082385,37.538558,"127.082385189457,37.5385583136667",서울특별시,광진구


In [10]:
yeongdong=['강릉시', '동해시', '속초시', '삼척시', '태백시', '고성군', '양양군']
yeongse=['원주시','춘천시','철원군','화천군','양구군','인제군','홍천군','평창군','정선군','영월군','횡성군']

In [11]:
print(set(weather['시도']))

{'강원영동', '전라남도', '경상북도', '경상남도', '제주특별자치도', '충청북도', '강원영서', '충청남도', '서울경기', '전라북도'}


In [12]:
print(set(all_sigun['시도']))

{'광주광역시', '강원도', '전라남도', '경상북도', '울산광역시', '경상남도', '제주특별자치도', '서울특별시', '충청북도', '인천광역시', '부산광역시', '충청남도', '세종특별자치시', '전라북도', '경기도', '대구광역시', '대전광역시'}


In [18]:
# 날씨 시도와 전국 시도 맞추기
w_dic={}
w_dic['인천광역시']='서울경기'
w_dic['대구광역시']='경상북도'
w_dic['부산광역시']='경상남도'
w_dic['울산광역시']='경상남도'
w_dic['서울특별시']='서울경기'
w_dic['대전광역시']='충청남도'
w_dic['경기도']='서울경기'
w_dic['광주광역시']='전라남도'
w_dic['세종특별자치시']='충청남도'
for sido in set(weather['시도']):
    if sido=='강원영동':
        for n in yeongdong:
            w_dic[n]=sido
    elif sido=='강원영서':
        for n in yeongse:
            w_dic[n]=sido
    else:
        w_dic[sido]=sido

In [19]:
print(w_dic)

{'인천광역시': '서울경기', '대구광역시': '경상북도', '부산광역시': '경상남도', '울산광역시': '경상남도', '서울특별시': '서울경기', '대전광역시': '충청남도', '경기도': '서울경기', '광주광역시': '전라남도', '세종특별자치시': '충청남도', '강릉시': '강원영동', '동해시': '강원영동', '속초시': '강원영동', '삼척시': '강원영동', '태백시': '강원영동', '고성군': '강원영동', '양양군': '강원영동', '전라남도': '전라남도', '경상북도': '경상북도', '경상남도': '경상남도', '제주특별자치도': '제주특별자치도', '충청북도': '충청북도', '원주시': '강원영서', '춘천시': '강원영서', '철원군': '강원영서', '화천군': '강원영서', '양구군': '강원영서', '인제군': '강원영서', '홍천군': '강원영서', '평창군': '강원영서', '정선군': '강원영서', '영월군': '강원영서', '횡성군': '강원영서', '충청남도': '충청남도', '서울경기': '서울경기', '전라북도': '전라북도'}


In [20]:
set(distance['month'])

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [27]:
# 전체 데이터 만들기
cols=['month','day','시도','시군구','철새수','최초발생거리','평균온도','일강수량','상대습도','위험도']
total_df=pd.DataFrame(columns=cols)
total_index_df=pd.DataFrame(columns=cols)
total=[]
total_index=[]
for month in tqdm(range(1,13)):
    start_date='2020-'+str(month)+'-01'
    start_date=datetime.datetime.strptime(start_date, '%Y-%m-%d')
    end_date=start_date+relativedelta(months=1)-datetime.timedelta(days=1)
    for day in tqdm(range(1,end_date.day+1)):
        for i in range(len(all_sigun)):
            q3=distance.query("month==@month").reset_index(drop=True)
            for j in range(len(q3)):
                temp={}
                index_temp={}
                temp['month']=month
                temp['day']=day
                sido=all_sigun.loc[i,'시도']
                temp['시도']=sido
                sigun=all_sigun.loc[i,'시군구']
                temp['시군구']=sigun
                index_temp['month']=month
                index_temp['day']=day
                index_temp['시도']=all_sigun.loc[i,'시도']
                index_temp['시군구']=all_sigun.loc[i,'시군구']

                q1=birds.query("month==@month and 시도==@sido").reset_index(drop=True)
                if len(q1)==0:
                    temp['철새수']=0
                    index_temp['철새수']=0
                else:
                    temp['철새수']=q1.loc[0,'철새수']
                    index_temp['철새수']=q1.loc[0,'risk']

                if temp['시도']=='강원도':
                    r_sigun=w_dic[sigun]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sigun").reset_index(drop=True)
                else:
                    r_sido=w_dic[sido]
                    q2=weather.query("month==@month and day==@day and 시도==@r_sido").reset_index(drop=True)
                temp['평균온도']=q2.loc[0,'avg_temp(C)']
                temp['일강수량']=q2.loc[0,'daily_rain(mm)']
                temp['상대습도']=q2.loc[0,'avg_r_humidity(%)']
                index_temp['평균온도']=q2.loc[0,'temp_index']
                index_temp['일강수량']=q2.loc[0,'rain_index']
                index_temp['상대습도']=q2.loc[0,'humidity_index']

                temp['최초발생거리']=q3.loc[j,'dist_range']
                index_temp['최초발생거리']=q3.loc[j,'risk']
                total.append(temp)
                total_index.append(index_temp)

  0%|                                                                                           | 0/31 [00:00<?, ?it/s]


  3%|██▋                                                                                | 1/31 [00:23<11:33, 23.13s/it]


  6%|█████▎                                                                             | 2/31 [00:46<11:12, 23.18s/it]


  7%|█████▌                                                                             | 2/30 [00:43<10:11, 21.84s/it]


 10%|████████▎                                                                          | 3/30 [01:15<11:18, 25.14s/it]


100%|███████████████████████████████████████████████████████████████████████████████| 12/12 [2:21:56<00:00, 709.73s/it]


In [28]:
total_df=total_df.append(total)
total_index_df=total_index_df.append(total_index)

In [29]:
total_df.to_csv("Train_data_set.csv", encoding='utf-8-sig', index=False)

In [30]:
total_index_df.to_csv("Train_risk_data_set.csv", encoding='utf-8-sig', index=False)

In [36]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2448189 entries, 0 to 2448188
Data columns (total 10 columns):
 #   Column  Dtype  
---  ------  -----  
 0   month   object 
 1   day     object 
 2   시도      object 
 3   시군구     object 
 4   철새수     object 
 5   최초발생거리  object 
 6   평균온도    float64
 7   일강수량    float64
 8   상대습도    float64
 9   위험도     float64
dtypes: float64(4), object(6)
memory usage: 205.5+ MB


In [35]:
total_df.query("시군구=='종로구'").reset_index()

,index,month,day,시도,시군구,철새수,최초발생거리,평균온도,일강수량,상대습도,위험도
0,0,1,1,서울특별시,종로구,126690,0,-1.8,0.0,64.2,NaN
1,1,1,1,서울특별시,종로구,126690,10,-1.8,0.0,64.2,NaN
2,2,1,1,서울특별시,종로구,126690,20,-1.8,0.0,64.2,NaN
3,3,1,1,서울특별시,종로구,126690,30,-1.8,0.0,64.2,NaN
4,4,1,1,서울특별시,종로구,126690,40,-1.8,0.0,64.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...
11822,2442210,12,31,서울특별시,종로구,136044,240,-3.2,0.1,62.0,NaN
11823,2442211,12,31,서울특별시,종로구,136044,250,-3.2,0.1,62.0,NaN
11824,2442212,12,31,서울특별시,종로구,136044,260,-3.2,0.1,62.0,NaN
11825,2442213,12,31,서울특별시,종로구,136044,270,-3.2,0.1,62.0,NaN


In [42]:
# 강수량 데이터는 의미없는것으로 보아서 독립변수로 지움
total_df.drop(['일강수량'], axis=1, inplace=True)
total_index_df.drop(['일강수량'], axis=1, inplace=True)

In [45]:
# 위험도 부여
total_index_df['위험도']=total_index_df['철새수']*0.3+total_index_df['최초발생거리']*0.5+total_index_df['평균온도']*0.12+total_index_df['상대습도']*0.08

In [46]:
total_df['위험도']=total_index_df['위험도']

In [48]:
total_df.to_csv("Train_data_set.csv", encoding='utf-8-sig', index=False)

In [49]:
total_index_df.to_csv("Train_risk_data_set.csv", encoding='utf-8-sig', index=False)